# Models

## SepConv1D

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
#from tensorflow.keras.regularizers import l2
#from keras.utils import np_utils



def SepConv1D(Chans = 8, Samples = 257, Filters = 32):
    eeg_input    = Input(shape = (Samples, Chans))
    padded       = ZeroPadding1D(padding = 4)(eeg_input)
    block1       = SeparableConv1D(Filters, 16, strides = 8,
                                   padding = 'valid',
                                   data_format = 'channels_last',
                                   kernel_initializer = 'glorot_uniform',
                                   bias_initializer = 'zeros',
                                   use_bias = True)(padded)
    block1       = Activation('tanh')(block1)
    flatten      = Flatten(name = 'flatten')(block1)
    prediction   = Dense(1, activation = 'sigmoid')(flatten)
    return Model(inputs = eeg_input, outputs = prediction, name='SepConv1D')

# Import Data

In [ ]:
import mne
filepath = "/workspace/data/EEG/data/"

ep1 = mne.read_epochs(filepath + "epochs/A01-epo.fif", preload=True)

In [3]:
labels = ep1.events[:, -1]

In [4]:
X = ep1.get_data() * 1000
y = labels-1

In [5]:
# take 50/20/20 percent of the data to train/validate/test
X_train      = X[0: int(0.6*len(y)),]
Y_train      = y[0: int(0.6*len(y)),]
X_validate   = X[int(0.6*len(y)) : int(0.8*len(y)) ,]
Y_validate   = y[int(0.6*len(y)) : int(0.8*len(y)) ,]
X_test       = X[int(0.8*len(y)) : len(y) ,]
Y_test       = y[int(0.8*len(y)) : len(y) ,]

In [8]:
kernels, chans, samples = 16, 8, 257


In [9]:

X_train      = X_train.reshape(X_train.shape[0], samples, chans)
X_validate   = X_validate.reshape(X_validate.shape[0], samples, chans)
X_test       = X_test.reshape(X_test.shape[0], samples, chans)

print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

X_train shape: (2520, 257, 8)
2520 train samples
840 test samples


In [10]:
model = SepConv1D()


import argparse
import sys
import numpy as np
#from SepConv1D import SepConv1D
from tensorflow.keras.callbacks import EarlyStopping
#from tensorflow import set_random_seed
from sklearn.model_selection import *
from utils import *
import tensorflow.keras.backend as K

def evaluate_subject_models(X_train, y_train, X_test, X_valid, y_valid, modelpath, n_filters = 32):
    """
    Trains and evaluates P300-CNNT for each subject in the P300 Speller database
    using repeated stratified K-fold cross validation.
    """

    #n_samples = 257
    #n_channels = 8

    model = SepConv1D()
    model.compile(optimizer = 'adam', loss = 'binary_crossentropy')

    es = EarlyStopping(monitor = 'val_loss', mode = 'min', patience = 50, restore_best_weights = True)
    history = model.fit(X_train,
                        y_train,
                        batch_size = 256,
                        epochs = 200,
                        validation_data = (X_valid, y_valid),
                        callbacks = [es],
                        class_weight = {0:1, 1:5})

    #model.load_weights('model.h5')
    predictions_single = model.predict(X_test)
    prediction_result = np.argmax(predictions_single[0])
    return predictions_single

2022-03-16 01:13:00.461770: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-03-16 01:13:01.221498: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 30988 MB memory:  -> device: 0, name: Tesla V100-SXM3-32GB, pci bus id: 0000:bc:00.0, compute capability: 7.0


In [ ]:
preds = evaluate_subject_models(X_train, Y_train, X_test, X_validate, Y_validate, "/workspace/data/EEG/models/", n_filters = 16)

In [ ]:
preds[preds > 0.5] = 1
preds[preds <= 0.5] = 0


In [ ]:
from sklearn.metrics import *
print("Test Accuracy: ", accuracy_score(Y_test, preds))
print("Test AUC: ", roc_auc_score(Y_test, preds))

print("Test f1 Score: ", f1_score(Y_test, preds))
print("Test Confusion matrix: ", confusion_matrix(Y_test, preds))
print(classification_report(Y_test, preds, labels= [0, 1]))


In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', 
              metrics = ['mse'])

# count number of parameters in the model
numParams    = model.count_params()

In [12]:

numParams

1441

In [13]:
from tensorflow.keras.callbacks import ModelCheckpoint

checkpointer = ModelCheckpoint(filepath='/workspace/data/EEG/models/checkpoint.h5', verbose=1,
                               save_best_only=True)

In [14]:
class_weights = {0:1, 1:5}

################################################################################
# fit the model. Due to very small sample sizes this can get
# pretty noisy run-to-run, but most runs should be comparable to xDAWN + 
# Riemannian geometry classification (below)
################################################################################
fittedModel = model.fit(X_train, Y_train, batch_size = 256, epochs = 500,
                        verbose = 2, callbacks=[checkpointer], validation_data=(X_validate, Y_validate), class_weight = class_weights)

Epoch 1/500


2022-03-16 01:13:22.504833: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8101
2022-03-16 01:13:22.981026: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory



Epoch 1: val_loss improved from inf to 0.00000, saving model to /workspace/data/EEG/models/checkpoint.h5
10/10 - 2s - loss: 0.0000e+00 - mse: 0.3068 - val_loss: 0.0000e+00 - val_mse: 0.3124 - 2s/epoch - 212ms/step
Epoch 2/500

Epoch 2: val_loss did not improve from 0.00000
10/10 - 0s - loss: 0.0000e+00 - mse: 0.2960 - val_loss: 0.0000e+00 - val_mse: 0.3252 - 78ms/epoch - 8ms/step
Epoch 3/500

Epoch 3: val_loss did not improve from 0.00000
10/10 - 0s - loss: 0.0000e+00 - mse: 0.3010 - val_loss: 0.0000e+00 - val_mse: 0.3395 - 73ms/epoch - 7ms/step
Epoch 4/500

Epoch 4: val_loss did not improve from 0.00000
10/10 - 0s - loss: 0.0000e+00 - mse: 0.3080 - val_loss: 0.0000e+00 - val_mse: 0.3527 - 68ms/epoch - 7ms/step
Epoch 5/500

Epoch 5: val_loss did not improve from 0.00000
10/10 - 0s - loss: 0.0000e+00 - mse: 0.3171 - val_loss: 0.0000e+00 - val_mse: 0.3646 - 67ms/epoch - 7ms/step
Epoch 6/500

Epoch 6: val_loss did not improve from 0.00000
10/10 - 0s - loss: 0.0000e+00 - mse: 0.3254 - val

In [15]:
model.load_weights('/workspace/data/EEG/models/checkpoint.h5')

In [16]:
import numpy as np
probs       = model.predict(X_test)
preds = probs
preds[preds > 0.5] = 1
preds[preds <= 0.5] = 0

acc         = np.mean(preds == Y_test.argmax(axis=-1))
print("Classification accuracy: %f " % (acc))

Classification accuracy: 0.448810 


In [17]:
preds

array([[0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [1.],
       [1.],
       [1.],
       [0.],
       [1.],
       [0.],
       [1.],
       [1.],
       [0.],
       [1.],
       [1.],
       [0.],
       [1.],
       [1.],
       [0.],
       [1.],
       [1.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [0.],
       [1.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [1.],
       [1.],

In [18]:
from sklearn.metrics import *
print("Test Accuracy: ", accuracy_score(Y_test, preds))
print("Test AUC: ", roc_auc_score(Y_test, preds))

print("Test f1 Score: ", f1_score(Y_test, preds))
print("Test Confusion matrix: ", confusion_matrix(Y_test, preds))
print(classification_report(Y_test, preds, labels= [0, 1]))


Test Accuracy:  0.5226190476190476
Test AUC:  0.4792857142857143
Test f1 Score:  0.22437137330754356
Test Confusion matrix:  [[381 319]
 [ 82  58]]
              precision    recall  f1-score   support

           0       0.82      0.54      0.66       700
           1       0.15      0.41      0.22       140

    accuracy                           0.52       840
   macro avg       0.49      0.48      0.44       840
weighted avg       0.71      0.52      0.58       840



In [29]:
type(X_train)

numpy.ndarray